In [1]:
import os

In [2]:
# Get iterable file list. Flattens hierarchical folder/file structure.

def GetFilesRecursive(path_root):
  final_list = []

  for directoryItem in dbutils.fs.ls(path_root):
    directoryItemPathClean = directoryItem.path.replace("%25", "%").replace("%25", "%")
    
    if directoryItem.isDir() == True:
      final_list = final_list + GetFilesRecursive(directoryItemPathClean)
    else:
      final_list.append(directoryItemPathClean)
  
  return final_list;

In [3]:
# Get iterable folder list. Flattens hierarchical folder structure.
# This can be useful for ingesting a set of Parquet folders, for example.

def GetFoldersRecursive(path_root):
  final_list = list()

  for directoryItem in dbutils.fs.ls(path_root):
    directoryItemPathClean = directoryItem.path.replace("%25", "%").replace("%25", "%")
    
    if directoryItem.isDir() == True:
      final_list.append(directoryItemPathClean)
      final_list = final_list + GetFoldersRecursive(directoryItemPathClean)
  
  return final_list;

In [4]:
# Delete Spark job residual files (_SUCCESS, _start*, _committed*) down the folder/file hierarchy

def CleanupSparkJobFiles(path_root):
  file_paths = GetFilesRecursive(path_root)
  
  for file_path in file_paths:
    # Get just the file name
    file_name = os.path.basename(file_path)
    # print(file_name)
    
    if file_name.startswith("_"):
      # Temp job file - delete it
      dbutils.fs.rm(file_path)
    # elif file_name.endswith(".parquet"):
      # Data file - no op
    # else:
      # Something else - no op